### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
import pickle

In [3]:
np.random.seed(42)

### Importing Data

This notebook is an extension of part 4 of this project. I read in all my datasets and wanted explore how well the models performed on them. Like before, the only model that ran was a Multinomial Naive Bayes, but a few datasets were able to run a Random Forest Classifier. I wanted to build a Streamlit App and will select the models that have the highest accuracy (and hopefully precision) scores. 

In [9]:
fiction_df = pd.read_csv('./data/fiction_sample.csv')

In [10]:
jvf_df = pd.read_csv('./data/jvf_sample.csv')

In [11]:
bio_df = pd.read_csv('./data/bio_sample.csv')

In [12]:
overall_df = pd.read_csv('./data/overall_sample.csv')

In [13]:
review_df = pd.read_csv('./data/review_sample.csv')

### Functions

* **best_params(pipeline, params, X_train, y_train)**: Reads in a pipeline, parameters, X_train, and y_train set that you've created, performs a GridSearchCV to find the best score and parameters through hypertuning. 
* **return_gs(pipeline, params, X_train, y_train)**: Returns GridSearch of a given pipeline and parameters
* **scores(gs, X_train, y_train, X_test, y_test)**: Using the returned gridsearch, the function will fit the model and perform a train-test-split to evaluate the R2 Train and Test scores.
* **predictions(pipeline, X_train, X_test, y_train)**: Returns predictions based on a pipeline and its model
* **classification_scores(model, y_test, y_pred)**: Using the predictions, it'll return recall, precision, f1, and accuracy scores for you to evaluate.

Note: Functions are reused from [my previous Subreddit project](https://git.generalassemb.ly/lisaliang/project-3.git)

In [14]:
def best_params(pipeline, params, X_train, y_train):
    gs = GridSearchCV(pipeline,
                      param_grid = params,
                      n_jobs=-1)

    gs.fit(X_train, y_train)
    return f'Best Score: {gs.best_score_}, Params: {gs.best_params_}'

In [15]:
def return_gs(pipeline, params, X_train, y_train):
    gs = GridSearchCV(pipeline,
                      param_grid = params,
                      n_jobs=-1)
    return gs

In [16]:
def scores(gs, X_train, y_train, X_test, y_test):
    gs.fit(X_train, y_train)
    return f'Train Score: {gs.score(X_train, y_train)}, Test Score: {gs.score(X_test, y_test)}'

In [17]:
def predictions(pipeline, X_train, X_test, y_train):
    pipeline.fit(X_train, y_train)
    prediction = pipeline.predict(X_test)
    
    return prediction

In [18]:
def classification_scores(model, y_test, y_pred):
    dataframe = pd.DataFrame(columns = ['Recall', 'Precision', 'F1', 'Accuracy'])
    
    recall = recall_score(y_test, y_pred, average = 'weighted')
    precision = precision_score(y_test, y_pred, average = 'weighted')
    f1 = f1_score(y_test, y_pred, average = 'weighted')
    accuracy = accuracy_score(y_test, y_pred)
    
    dataframe.loc[model] = [recall, precision, f1, accuracy]
    
    return dataframe

* **my_lemmatizer(text)**: This function lemmatizes inputted text to their dictionary forms. It adds conditions to filter out words with apostrophes or digits so they are done as accurately as possible.

Additional: We created a list of English stopwords, contractions, and numbers for the model to remove while it's iterating through the text. These attributes were seen as not adding significance in helping the model distinguish book titles.

In [19]:
def my_lemmatizer(text):
    wnet = WordNetLemmatizer()
    # exclude words with apostrophes and numbers
    return [wnet.lemmatize(w) for w in text.split() if "'" not in w and not w.isdigit()]

In [20]:
wnet = WordNetLemmatizer()
lem_stopwords = [wnet.lemmatize(w) for w in stopwords.words('english')]

contractions = ['ve', 't', "'s'", 'd', 'll', 'm', 're']
lem_contractions = [wnet.lemmatize(contraction) for contraction in contractions]

numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
lem_numbers = [wnet.lemmatize(num) for num in numbers]

lem_stopwords = lem_stopwords + lem_contractions + lem_numbers

### Fiction (MNB)

In [16]:
X_fiction = fiction_df['description']
y_fiction = fiction_df['Title']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_fiction, y_fiction, random_state=42)

In [18]:
fiction_pipe = Pipeline([
    ('tf', TfidfVectorizer(stop_words = lem_stopwords, 
                           tokenizer = my_lemmatizer,
                           token_pattern = None,
                           max_features = 500)),
    ('mnb', MultinomialNB(alpha = 0.5))
])

In [19]:
fiction_params = {
    'tf__min_df': [0.05, 0.1],
    'tf__max_df': [0.5],
    'tf__ngram_range': [(1,1)],
    'mnb__alpha': [0.05, 0.1],
    'mnb__fit_prior': [False]
}

In [20]:
fiction_gs = return_gs(fiction_pipe, fiction_params, X_train, y_train)

In [21]:
scores(fiction_gs, X_train, y_train, X_test, y_test)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


'Train Score: 0.8729866666666667, Test Score: 0.81104'

In [22]:
fiction_pred = predictions(fiction_pipe, X_train, X_test, y_train)

In [23]:
classification_scores('Fiction', y_test, fiction_pred)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Recall,Precision,F1,Accuracy
Fiction,0.67144,0.515462,0.57021,0.67144


In [38]:
with open('fiction_pipe.pkl', 'wb') as f:
    pickle.dump(fiction_pipe, f)

### Juvenile Fiction (Random Forest)

In [36]:
X_jvf = jvf_df['description']
y_jvf = jvf_df['Title']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_jvf, y_jvf, random_state=42)

In [38]:
jvf_pipe = Pipeline([
    ('tf', TfidfVectorizer(stop_words = lem_stopwords, 
                           tokenizer = my_lemmatizer,
                           token_pattern = None,
                           max_features = 1_000)),
    ('rfc', RandomForestClassifier(max_features = 1_000))
])

In [39]:
jvf_params = {
    'tf__min_df': [0.05, 0.1],
    'tf__max_df': [0.5],
    'tf__ngram_range': [(1,1)]
}

In [40]:
jvf_gs = return_gs(jvf_pipe, jvf_params, X_train, y_train)

In [41]:
scores(jvf_gs, X_train, y_train, X_test, y_test)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


'Train Score: 0.7850666666666667, Test Score: 0.75784'

In [42]:
jvf_pred = predictions(jvf_pipe, X_train, X_test, y_train)

In [43]:
classification_scores('Juvenile Fiction', y_test, jvf_pred)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Recall,Precision,F1,Accuracy
Juvenile Fiction,0.84768,0.774386,0.801599,0.84768


In [44]:
with open('jvf_pipe.pkl', 'wb') as f:
    pickle.dump(jvf_pipe, f)

### Biography & Autobiography (Random Forest)

In [21]:
X_bio = bio_df['description']
y_bio = bio_df['Title']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_bio, y_bio, random_state=42)

In [23]:
bio_pipe = Pipeline([
    ('tf', TfidfVectorizer(stop_words = lem_stopwords, 
                           tokenizer = my_lemmatizer,
                           token_pattern = None,
                           max_features = 1_000)),
    ('rfc', RandomForestClassifier(max_features = 1_000))
])

In [24]:
bio_params = {
    'tf__min_df': [0.05, 0.1],
    'tf__max_df': [0.5],
    'tf__ngram_range': [(1,1)],
}

In [25]:
bio_gs = return_gs(bio_pipe, bio_params, X_train, y_train)

In [26]:
scores(bio_gs, X_train, y_train, X_test, y_test)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


'Train Score: 0.90608, Test Score: 0.89288'

In [27]:
bio_pred = predictions(bio_pipe, X_train, X_test, y_train)

In [28]:
classification_scores('Biography & Autobiography', y_test, bio_pred)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Recall,Precision,F1,Accuracy
Biography & Autobiography,0.91488,0.875228,0.888892,0.91488


In [29]:
with open('bio_rfc.pkl', 'wb') as f:
    pickle.dump(bio_pipe, f)